# Training DNN
- tricky vanishing gradients problem or related exploding gradients problem
- Don't have enough training data for such a large network, or too costly to label
- Training may be extremely slow
- A model ith million of parameters that risk to overfitting the training set, especially if there are not enough training instances or if they are too noisy.

## 1. Vanishing/Exploding Gradients Problems
Gradients often get smaller and smaller as the algorithm progresses down to the lower layers (vanishing), or gradients can grow bigger and bigger until layers get insanely large weight updates and the algorithm diverges (Exploding).

```
+---------------+-------------------------------+---------------------+
|Initialization |  Activation function          | $\sigma^2$ (Normal) |
+---------------+-------------------------------+---------------------+
| Glorot        | None, tanh, logistic, softmax | $1/fan_{avg}$       |
| He            | ReLU, variants                | $2/fan_{in}$        |
| LeCun         | SELU                          | $1/fan_{in}$        |
+---------------+-------------------------------+---------------------+
```
Keras uses Glorot initialization (default) with a uniform distribution. When creating a layer, we can change it to He initialization by kernel_initializer="he_uniform" or "he_normal".

If you want He initialization with a uniform distribution but based on fan avg rather than fan in , you can use the VarianceScaling initializer like this:
``` 
he_avg_init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')
keras.layers.Dense(10, activation="sigmoid", kernel_initializer=he_avg_init)
```
He initialization: ``` keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")```

If you want He initialization with a uniform distribution but based on fan avg rather than fan in , you can use the VarianceScaling initializer like this:
```
he_avg_init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')
keras.layers.Dense(10, activation="sigmoid", kernel_initializer=he_avg_init)
```

## 1.1. Nonsaturating Activation Functions
The dying ReLUs: during training, some neurons effectively "die", meaning they stop outputting anything other than 0. 
- Solve this problem, the leaky ReLU: $leakyReLU_{\alpha}(z) = max(\alpha z, z)$. The hyperparameter $\alpha$ defines how much function leak. 
- ELU (Exponential Linear Unit): like a ReLU function but, takes negative when z < 0, it has nonzero gradient for z < 0 (avoid dead neuron problem). But ELU is slower to compute than ReLU.
- SELU (Scaled ELU): the network will self-normalize (mean=0, std=1). Using constrains: input features must be standardized, hidden layer's weight must be initialized with LeCun normal initialization (kernel_initializer="lecun_normal"), the network's architecture must be Sequential(). 

Which activation function should use? in general: SELU > ELU > leaky ReLU (and its variants) > ReLU > tanh > logistic.

Using SELU: ``` keras.layers.Dense(10, activation="selu", kernel_initializer="lecun_normal") ```

Implementing ELU in TensorFlow is trivial, just specify the activation function when building each layer: ```keras.layers.Dense(10, activation="elu") ```

Sample of training model with *Leaky ReLU* activation function:
```
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])
```

## 1.2. Batch Normalization (BN)
The technique consists of adding an operation in the model just before or after activation function of each hidden layer. This operation is simply zero-center and normalization each input, then scales and shifts the result using two new parameter vectors per layer: one for scaling, the other for shifting. In other words, the operation lets the model learn the optimal scale and mean of each of the layer's input.

Implement of BN to create the model:
```
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])
```
Let’s look at the parameters of the first BN layer. Two are trainable (by backpropagation), and two are not:
```
>>> [(var.name, var.trainable) for var in model.layers[1].variables]
        [('batch_normalization_v2/gamma:0', True),
        ('batch_normalization_v2/beta:0', True),
        ('batch_normalization_v2/moving_mean:0', False),
        ('batch_normalization_v2/moving_variance:0', False)]
```
 **Notice:**

> Adding BN layer before the activation functions, rather than after. To add the BN function before the activation functions, we must remove the activation function from the hidden layers and add them as separate layer after BN layer. Moreover, the BN layer includes on offset parameter per input, we can remove the bias term from the previous hidden layer by using use_bias=False. Sample code:
```
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(10, activation="softmax")
])
```

## 1.3. Gradient Clipping
Mitigate the exploding gradients problem is to clip the gradients during backpropagation so that they never exceed some threshold. 

This technique is most often used in the recurrent neural networks, BN is tricky to use in RNN.

In Keras, implementing Gradient Clipping is just a matter of setting the clipvalue or clipnorm argument when creating an optimizer, like this:
```
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer)
```
This optimizer will clip every component of the gradient vector to a value between –1.0 and 1.0. This means that all the partial derivatives of the loss (with regard to each and every trainable parameter) will be clipped between –1.0 and 1.0. The threshold is a hyperparameter you can tune. Note that it may change the orientation of the gradient vector. Using *clipnorm* instead of *clipvalue* for not change the direction of the gradient vector.

## 2. Reusing Pretrained layers
Reusing a existing neural network that accomplishes a similar task, and then reuse the lower layers of this network. This is called *transfer learning*.

> If the input pictures of your new task don’t have the same size as the ones used in the original task, you will usually have to add a preprocessing step to resize them to the size expected by the original model. More generally, transfer learning will work best when the inputs have similar low-level features.

The output layer of the original model should usually be replaced because it is most likely not useful at all for the new task, and it may not even have the right number of outputs for the new task.

First, you need to load model A and create a new model based on that model’s layers. Let’s reuse all the layers except for the output layer:
```
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))
```
Note that model_A and model_B_on_A now share some layers. When you train model_B_on_A , it will also affect model_A . If you want to avoid that, you need to clone model_A before you reuse its layers. To do this, you clone model A’s architecture with clone_model() , then copy its weights (since clone_model() does not clone the weights):
```
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
```
## 2.1. Unsupervised Pretraining
This technique is called *greedy layer-wise pretraining*. They would first train an unsupervised model with a single layer, typically an RBM, then they would freeze that layer and add another one on top of it, then train the model again (effectively just training the new layer), then freeze the new layer and add another layer on top of it, train the model again, and so on. Nowadays, things are much simpler: people generally train the full unsupervised model in one shot (i.e., in Figure 11-5, just start directly at step three) and use autoencoders or GANs rather than RBMs.

<img src="images/deeplearningUnlabeledmodel.png" width="442" height="442" style="vertical-align:middle;"/>

*Figure 11-5. In unsupervised training, a model is trained on the unlabeled data (or on all the data) using an unsupervised learning technique, then it is fine-tuned for the finaltask on the labeled data using a supervised learning technique; the unsupervised part may train one layer at a time as shown here, or it may train the full model directly*

## 2.2. Pretraining on an Auxiliary task
One last option is to train a first neural network on an auxiliary task for which you can easily obtain or generate labeled training data, then reuse the lower layers of that network for your actual task. The first neural network’s lower layers will learn feature detectors that will likely be reusable by the second neural network.

Self-supervised learning is when you automatically generate the labels from the data itself, then you train a model on the resulting“labeled” dataset using supervised learning techniques. Since this approach requires no human labeling whatsoever, it is best classified as a form of unsupervised learning.

## 3. Faster Optimizers
Speed up training very large NN with some techniques:
- Applying a good initialization strategy for the connection weights.
- Using a good activation function
- Using Batch Normalization
- Reusing parts of a pretrained network (possibly build on an auxiliary task or using unsupervised learning)

Other techniques:
- momentum optimization
- Nesterov accelerated gradient
- AdaGrad
- RMSProp
- Adam and Nadam optimization

## 3.1. Momentum Optimization
GD takes small steps down the slope, so the algorithm will take much more time to reach the bottom. Momentum opt cares about what previous gradients were: at each iteration, it subtracts the local gradient from the momentum vector m, and update the weight by adding this momentum vector. Value Range: 0 (high friction) to 1 (no friction). *A typical momentum value is 0.9*
```
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
```
## 3.2. Nesterov Accelerated Gradient
Measure the gradient of the cost function not at the local position $ \theta $ but slightly ahead in the direction of the momentum, at $ \theta + \beta m$

NAG is faster than regular momentum optimization.
```
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True) 
```
## 3.3. AdaGrad
The AdaGrad algorithm achieves this correction by scaling down the gradient vector along the steepest dimensions. AdaGrad frequently performs well for simple quadratic problems, but it often stops too early when training neural networks. The learning rate gets scaled down so much that the algorithm ends up stopping entirely before reaching the global optimum. So even though Keras has an Adagrad optimizer, you should not use it to train deep neural networks (it may be efficient for simpler tasks such as Linear Regression, though). Still, understanding AdaGrad is helpful to grasp the other adaptive learning rate optimizers.
```
optimizer = keras.optimizers.Adagrad(learning_rate=0.001)
```
## 3.4. RMSProp
Better than AdaGrad and fixes its problems.
```
optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
```
## 3.5. Adam Optimization
Adam (adaptive moment estimation), Since Adam is an adaptive learning rate algorithm (like AdaGrad and RMSProp), it requires less tuning of the learning rate hyperparameter η. You can often use the default value η = 0.001, making Adam even easier to use than Gradient Descent. 
```
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
```
## 3.6. Adamax Optimization
AdaMax, introduced in the same paper as Adam, replaces the ℓ_2 norm with the ℓ_∞ norm (a fancy way of saying the max)
```
optimizer = keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
```
## 3.7. Nadam Optimization
Nadam optimization is Adam optimization plus the Nesterov trick, so it will often converge slightly faster than Adam.
```
optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
```
## 4. Training Sparse models
All the optimization algorithms  just presented produce dense model (most parameters will be nonzero). Sparse models are faster at runtime, less memory using (most parameters are zero). To use Sparse models, applying l_1 or l_2 regularization (in Lasso Regression)

## 5. Learning Rate Scheduling
Find a good learning rate. There are some strategies are called learning schedules:
- Power scheduling: learning rate drops at each step
- Exponential scheduling: the learning rate will gradually drop by a factor of 10 every s steps.
- Piecewise constant scheduling: using a constant learning rate for number of epochs (eg., learning_rate=0.1 for 5 epochs)
- Performance scheduling: measure the validation error every N steps
- 1cycle scheduling: 1cycle starts by increasing the initial learning rate η_0 , growing linearly up to η_1 halfway through training. Then it decreases the learning rate linearly down to η_0 again during the second half of training, finishing the last few epochs by dropping the rate down by several orders of magnitude (still linearly). The maximum learning rate η_1 is chosen using the same approach we used to find the optimal learning rate, and the initial learning rate η_0 is chosen to be roughly 10 times lower.

Implementing power scheduling in Keras by setting the decay hyperparameter when creating an optimizer:
```
optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)
```
The *decay* is the inverse of s (number of steps) and Keras assume that c is equal to 1.

> Customize learning rate scheduling: https://github.com/ageron/handson-ml2/blob/master/11_training_deep_neural_networks.ipynb

## 6. Avoiding Overfitting through Regularization
- Early stopping technique is one of the best regularization, Batch Normalization solve the unstable gradients problem.
### 6.1. l_1 and l_2 Regularization
We can use ℓ 2 regularization to constrain a neural network’s connection weights, and/or ℓ 1 regularization if you want a sparse model (with many weights equal to 0). Here is how to apply ℓ 2 regularization to a Keras layer’s connection weights, using a regularization factor of 0.01:
```
layer = keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))
```

Customize regularization with Python's functools.partial() function, which is quickly creating a thin wrapper for any callable:
```
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
        activation="elu",
        kernel_initializer="he_normal",
        kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax",
    kernel_initializer="glorot_uniform")
])
```

### 6.2. Dropout
The most popular regularization techniques for deep neural networks. It is a fairly simple algorithm: at every training step, every neuron (including the input neurons, but always excluding the output neurons) has a probability p of being temporarily “dropped out,” meaning it will be entirely ignored during this training step, but it may be active during the next step (see Figure 11-9). The hyperparameter p is called the dropout rate, and it is typically set between 10% and 50%: closer to 20–30% in recurrent neural nets (see Chapter 15), and closer to 40–50% in convolutional neural networks (see Chapter 14). After training, neurons don’t get dropped anymore. And that’s all (except for a technical detail we will discuss momentarily).

The following code applies dropout regularization before every Dense layer, using a dropout rate of 0.2:
```
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
```

> Since dropout is only active during training, comparing the training loss and the validation loss can be misleading. In particular, a model may be overfitting the training set and yet have similar training and validation losses. So make sure to evaluate the training loss without dropout (e.g., after training).

### 6.3. Monte Carlo (MC) Dropout
(Hand-on ML with scikit-learn, keras, and tensorflow, page 368)
### 6.4. Max-Norm Regularization

for each neuron, it constrains the weights w of the incoming connections such that ∥ w ∥_2 ≤ r, where r is the max-norm hyperparameter and ∥ · ∥_2 is the ℓ_2 norm.

To implement max-norm regularization in Keras, set the kernel_constraint argument of each hidden layer to a max_norm() constraint with the appropriate max value, like this:
```
keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal", kernel_constraint=keras.constraints.max_norm(1.))
```

# Summary 


In [1]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# show all available initializers
[name for name in dir(keras.initializers) if not name.startswith('_')]

['Constant',
 'GlorotNormal',
 'GlorotUniform',
 'HeNormal',
 'HeUniform',
 'Identity',
 'Initializer',
 'LecunNormal',
 'LecunUniform',
 'Ones',
 'Orthogonal',
 'RandomNormal',
 'RandomUniform',
 'TruncatedNormal',
 'VarianceScaling',
 'Zeros',
 'constant',
 'deserialize',
 'get',
 'glorot_normal',
 'glorot_uniform',
 'he_normal',
 'he_uniform',
 'identity',
 'lecun_normal',
 'lecun_uniform',
 'ones',
 'orthogonal',
 'random_normal',
 'random_uniform',
 'serialize',
 'truncated_normal',
 'variance_scaling',
 'zeros']

In [3]:
[m for m in dir(keras.activations) if not m.startswith("_")]

['deserialize',
 'elu',
 'exponential',
 'gelu',
 'get',
 'hard_sigmoid',
 'linear',
 'relu',
 'selu',
 'serialize',
 'sigmoid',
 'softmax',
 'softplus',
 'softsign',
 'swish',
 'tanh']

In [4]:
[m for m in dir(keras.layers) if "relu" in m.lower()]

['LeakyReLU', 'PReLU', 'ReLU', 'ThresholdedReLU']

# Reusing Pretrained layers
Let's split the fashion MNIST training set in two:

- X_train_A: all images of all items except for sandals and shirts (classes 5 and 6).
- X_train_B: a much smaller training set of just the first 200 images of sandals or shirts.

The validation set and the test set are also split this way, but without restricting the number of images.

We will train a model on set A (classification task with 8 classes), and try to reuse it to tackle set B (binary classification). We hope to transfer a little bit of knowledge from task A to task B, since classes in set A (sneakers, ankle boots, coats, t-shirts, etc.) are somewhat similar to classes in set B (sandals and shirts). However, since we are using Dense layers, only patterns that occur at the same location can be reused (in contrast, convolutional layers will transfer much better, since learned patterns can be detected anywhere on the image, as we will see in the CNN chapter).

In [5]:
# load data set
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [6]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [7]:
# define the model A
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

model_A.summary()

2022-04-10 23:34:56.118559: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 23:34:56.183095: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 23:34:56.183267: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 50)                5050      
                                                                 
 dense_3 (Dense)             (None, 50)                2550      
                                                                 
 dense_4 (Dense)             (None, 50)                2550      
                                                                 
 dense_5 (Dense)             (None, 8)                 4

2022-04-10 23:34:56.184856: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-10 23:34:56.186543: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 23:34:56.186702: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-10 23:34:56.186820: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [8]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A), verbose=0)

2022-04-10 23:36:22.712938: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [9]:
model_A.save('models/my_model_A.h5')

In [10]:
# define model B
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

model_B.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_6 (Dense)             (None, 300)               235500    
                                                                 
 dense_7 (Dense)             (None, 100)               30100     
                                                                 
 dense_8 (Dense)             (None, 50)                5050      
                                                                 
 dense_9 (Dense)             (None, 50)                2550      
                                                                 
 dense_10 (Dense)            (None, 50)                2550      
                                                                 
 dense_11 (Dense)            (None, 1)                

In [11]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B), verbose=0)

In [12]:
# load model A
model_A = keras.models.load_model('models/my_model_A.h5')
# create new clone of model A and reuse lower layers of A to B
# when training B on A and A is not update 
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())  # reset the weights of clone_A by A
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [13]:
# because model_B_on_A for task B, that will make some large error when training with few epochs (output layer was initialized randomly).
# We train it with few epochs which setting attribute trainable=False
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B), verbose=0)

history.history['loss']

[0.9109138250350952, 0.841697096824646, 0.7791298031806946, 0.7228350043296814]

In [14]:
# now, setting back the trainable=True and train it again
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

history.history['loss'][-1]

Epoch 1/16
7/7 [==============================] - 0s 21ms/step - loss: 0.5641 - accuracy: 0.7250 - val_loss: 0.4369 - val_accuracy: 0.8235
Epoch 2/16
7/7 [==============================] - 0s 7ms/step - loss: 0.3583 - accuracy: 0.8800 - val_loss: 0.3229 - val_accuracy: 0.8935
Epoch 3/16
7/7 [==============================] - 0s 7ms/step - loss: 0.2639 - accuracy: 0.9350 - val_loss: 0.2589 - val_accuracy: 0.9280
Epoch 4/16
7/7 [==============================] - 0s 7ms/step - loss: 0.2094 - accuracy: 0.9600 - val_loss: 0.2195 - val_accuracy: 0.9462
Epoch 5/16
7/7 [==============================] - 0s 7ms/step - loss: 0.1745 - accuracy: 0.9650 - val_loss: 0.1903 - val_accuracy: 0.9544
Epoch 6/16
7/7 [==============================] - 0s 7ms/step - loss: 0.1491 - accuracy: 0.9750 - val_loss: 0.1700 - val_accuracy: 0.9594
Epoch 7/16
7/7 [==============================] - 0s 7ms/step - loss: 0.1310 - accuracy: 0.9750 - val_loss: 0.1538 - val_accuracy: 0.9645
Epoch 8/16
7/7 [=================

0.06381265074014664

In [15]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 838us/step - loss: 0.1015 - accuracy: 0.9795


[0.10145364701747894, 0.9794999957084656]

In [16]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 856us/step - loss: 0.0829 - accuracy: 0.9830


[0.08294575661420822, 0.9829999804496765]

In [18]:
# the score between 2 models
(1 - 0.9795) / (1 - 0.983)

1.2058823529411733

# exercise
Deep learning on CIFAR10, The dataset is composed of 60,000 32 × 32–pixel color images (50,000 for training, 10,000 for testing) with 10 classes

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os

%matplotlib inline

In [3]:
# a. Build 20 hidden layers of 100 neurons each, use He initialization ELU activation
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'))
    
# b. the output for 10 classes with 10 neurons, softmax activation
model.add(keras.layers.Dense(10, activation='softmax'))

## Remember to search for the right learning rate each time you change the model's architecture or hyperparameters.

Let's use a Nadam optimizer with a learning rate of 5e-5. I tried learning rates 1e-5, 3e-5, 1e-4, 3e-4, 1e-3, 3e-3 and 1e-2, and I compared their learning curves for 10 epochs each (using the TensorBoard callback, below). The learning rates 3e-5 and 1e-4 were pretty good, so I tried 5e-5, which turned out to be slightly better.

In [5]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-5)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dense_21 (Dense)            (None, 100)               307300    
                                                                 
 dense_22 (Dense)            (None, 100)               10100     
                                                                 
 dense_23 (Dense)            (None, 100)               10100     
                                                                 
 dense_24 (Dense)            (None, 100)               10100     
                                                                 
 dense_25 (Dense)            (None, 100)               10100     
                                                                 
 dense_26 (Dense)            (None, 100)              

In [6]:
# load CIFAR10 dataset
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

170508288/170498071 [==============================] - 51s 0us/step


In [7]:
# Now, we can create the callbacks and train the model
early_stop_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("models/my_cifar10_checkpoint.h5", save_best_only=True)
run_index = 1   # increase when training 
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stop_cb, model_checkpoint_cb, tensorboard_cb]

In [9]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=callbacks, verbose=0)

2022-04-11 21:40:55.948048: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [10]:
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 0s 1ms/step - loss: 1.5629 - accuracy: 0.4750


[1.562867522239685, 0.4749999940395355]

In [12]:
model = keras.models.load_model('models/my_cifar10_checkpoint.h5')
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 0s 1ms/step - loss: 1.5237 - accuracy: 0.4752


[1.5237149000167847, 0.47519999742507935]

### c.
Now, adding Batch Normalization and compare the learning curve. Is it converging faster than before? Does it produce a better model? How does it affect training speed?

The code below is very similar to the code above, with a few changes:

- I added a BN layer after every Dense layer (before the activation function), except for the output layer. I also added a BN layer before the first hidden layer.
- I changed the learning rate to 5e-4. I experimented with 1e-5, 3e-5, 5e-5, 1e-4, 3e-4, 5e-4, 1e-3 and 3e-3, and I chose the one with the best validation performance after 20 epochs.
- I renamed the run directories to runbn* and the model file name to my_cifar10_bn_model.h5

In [13]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer='he_normal'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation('elu'))

model.add(keras.layers.Dense(10, activation='softmax'))

optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint('models/my_cifar10_bn_checkpoint.h5', save_best_only=True)
run_index = 1
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=callbacks, verbose=0)

In [14]:
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 0s 2ms/step - loss: 1.4163 - accuracy: 0.5326


[1.4162912368774414, 0.5325999855995178]

In [15]:
model = keras.models.load_model('models/my_cifar10_bn_checkpoint.h5')
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 1s 2ms/step - loss: 1.3245 - accuracy: 0.5430


[1.3245311975479126, 0.5429999828338623]

### d.
Exercise: Try replacing Batch Normalization with SELU, and make the necessary adjustements to ensure the network self-normalizes (i.e., standardize the input features, use LeCun normal initialization, make sure the DNN contains only a sequence of dense layers, etc.).

In [17]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="lecun_normal", activation="selu"))
    
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=7e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("models/my_cifar10_selu_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_selu_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid), callbacks=callbacks, verbose=0)

#model = keras.models.load_model("models/my_cifar10_selu_model.h5")
model.evaluate(X_valid_scaled, y_valid)

157/157 [==============================] - 0s 1ms/step - loss: 1.6657 - accuracy: 0.5138


[1.6656558513641357, 0.5138000249862671]

In [18]:
model = keras.models.load_model("models/my_cifar10_selu_model.h5")
model.evaluate(X_valid_scaled, y_valid)

157/157 [==============================] - 0s 1ms/step - loss: 1.4844 - accuracy: 0.4950


[1.4844242334365845, 0.4950000047683716]

### e.
Exercise: Try regularizing the model with alpha dropout. Then, without retraining your model, see if you can achieve better accuracy using MC Dropout.

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))

model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("models/my_cifar10_alpha_dropout_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_alpha_dropout_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks, verbose=0)

model = keras.models.load_model("models/my_cifar10_alpha_dropout_model.h5")
model.evaluate(X_valid_scaled, y_valid)

The model reaches 48.9% accuracy on the validation set. That's very slightly better than without dropout (47.6%). With an extensive hyperparameter search, it might be possible to do better (I tried dropout rates of 5%, 10%, 20% and 40%, and learning rates 1e-4, 3e-4, 5e-4, and 1e-3), but probably not much better in this case.

Let's use MC Dropout now. We will need the MCAlphaDropout class we used earlier, so let's just copy it here for convenience:

In [ ]:
class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [ ]:
# create a new model, identical to the one we just trained (with the same weights), but with MCAlphaDropout dropout layers instead of AlphaDropout layers:
mc_model = keras.models.Sequential([
    MCAlphaDropout(layer.rate) if isinstance(layer, keras.layers.AlphaDropout) else layer
    for layer in model.layers
])

In [ ]:
# add a couple utility functions
def mc_dropout_predict_probas(mc_model, X, n_samples=10):
    Y_probas = [mc_model.predict(X) for sample in range(n_samples)]
    return np.mean(Y_probas, axis=0)

def mc_dropout_predict_classes(mc_model, X, n_samples=10):
    Y_probas = mc_dropout_predict_probas(mc_model, X, n_samples)
    return np.argmax(Y_probas, axis=1)

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

y_pred = mc_dropout_predict_classes(mc_model, X_valid_scaled)
accuracy = np.mean(y_pred == y_valid[:, 0])
accuracy

### f.
Exercise: Retrain your model using 1cycle scheduling and see if it improves training speed and model accuracy.

In [19]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))

model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.SGD(learning_rate=1e-3)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [24]:
## 1cycle scheduling
import math


K = keras.backend

class ExponentialLearningRate(keras.callbacks.Callback):
    def __init__(self, factor):
        self.factor = factor
        self.rates = []
        self.losses = []
    def on_batch_end(self, batch, logs):
        self.rates.append(K.get_value(self.model.optimizer.learning_rate))
        self.losses.append(logs["loss"])
        K.set_value(self.model.optimizer.learning_rate, self.model.optimizer.learning_rate * self.factor)

def find_learning_rate(model, X, y, epochs=1, batch_size=32, min_rate=10**-5, max_rate=10):
    init_weights = model.get_weights()
    iterations = math.ceil(len(X) / batch_size) * epochs
    factor = np.exp(np.log(max_rate / min_rate) / iterations)
    init_lr = K.get_value(model.optimizer.learning_rate)
    K.set_value(model.optimizer.learning_rate, min_rate)
    exp_lr = ExponentialLearningRate(factor)
    history = model.fit(X, y, epochs=epochs, batch_size=batch_size,
                        callbacks=[exp_lr])
    K.set_value(model.optimizer.learning_rate, init_lr)
    model.set_weights(init_weights)
    return exp_lr.rates, exp_lr.losses

def plot_lr_vs_loss(rates, losses):
    plt.plot(rates, losses)
    plt.gca().set_xscale('log')
    plt.hlines(min(losses), min(rates), max(rates))
    plt.axis([min(rates), max(rates), min(losses), (losses[0] + min(losses)) / 2])
    plt.xlabel("Learning rate")
    plt.ylabel("Loss")

In [ ]:
batch_size = 128
rates, losses = find_learning_rate(model, X_train_scaled, y_train, epochs=1, batch_size=batch_size)
plot_lr_vs_loss(rates, losses)
plt.axis([min(rates), max(rates), min(losses), (losses[0] + min(losses)) / 1.4])

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))

model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.SGD(learning_rate=1e-2)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [ ]:
n_epochs = 15
onecycle = OneCycleScheduler(math.ceil(len(X_train_scaled) / batch_size) * n_epochs, max_rate=0.05)
history = model.fit(X_train_scaled, y_train, epochs=n_epochs, batch_size=batch_size,
                    validation_data=(X_valid_scaled, y_valid),
                    callbacks=[onecycle])